In [1]:
import re
import os
import time
import numpy as np
import torch
from torchvision import transforms
from PIL import Image as Image2
from collections import namedtuple
from IPython.display import display, Image

from ipywidgets import interact, interactive
from transformernet import TransformerNet

In [2]:
# load image with optional size or scale
def load_image(filename, size=None, scale=None):
    img = Image2.open(filename)
    if size is not None:
        img = img.resize((size, size), Image2.ANTIALIAS)
    elif scale is not None:
        img = img.resize((int(img.size[0] / scale), int(img.size[1] / scale)), Image2.ANTIALIAS)
    return img

# save image
def save_image(filename, data):
    img = data.clone().clamp(0, 255).numpy()
    img = img.transpose(1, 2, 0).astype("uint8")
    img = Image2.fromarray(img)
    img.save(filename)

# return gram matrix of input layer outputs
def gram_matrix(y):
    (b, ch, h, w) = y.size()
    features = y.view(b, ch, w * h)
    features_t = features.transpose(1, 2)
    # compute gram product
    gram = features.bmm(features_t) / (ch * h * w)
    return gram


def normalize_batch(batch):
    # normalize using imagenet mean and std. normalize so we can easily put it in a nn.Sequential
    # allows to work with image Tenor of shape
    mean = batch.new_tensor([0.485, 0.456, 0.406]).view(-1, 1, 1)
    std = batch.new_tensor([0.229, 0.224, 0.225]).view(-1, 1, 1)
    batch = batch.div_(255.0)
    return (batch - mean) / std


def stylize(style, content):
    # check if we can use a gpu
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    #stylize image
    content_image = load_image(content + ".jpg", size=None)
    content_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.mul(255))
    ])
    content_image = content_transform(content_image)
    content_image = content_image.unsqueeze(0).to(device)
    with torch.no_grad():
        style_model = TransformerNet()
        # INPUT, model you want to stylize with
        state_dict = torch.load("models/" + style + ".model")
        # remove saved deprecated running_* keys in InstanceNorm from the checkpoint
        for k in list(state_dict.keys()):
            if re.search(r'in\d+\.running_(mean|var)$', k):
                del state_dict[k]
        style_model.load_state_dict(state_dict)
        style_model.to(device)
        output = style_model(content_image).cpu()
    save_image(content + '_' + style + ".jpg", output[0])
    #i_style = Image(filename='styles/' + style + '.jpg')
    #i_content = Image(filename=content + '.jpg')
    i_result = Image(filename=content + '_' + style + ".jpg")
    #display(i_style)
    #display(i_content)
    display(i_result)
def display_style(x):
    i = Image(filename='styles/' + x + '.jpg')
    display(i)
    
def display_content(x):
    i = Image(filename=x + '.jpg')
    display(i)
    

In [3]:
style_suite = ['bullfight', '2500_bullfight', '5000_bullfight', '10000_bullfight', '20000_bullfight', '40000_bullfight', 'candy', 'mosaic','digbychildren','funeraryslab','mosaic','mounthood','plumblossoms','streetscene','tarzana','udnie']
content_suite = ['law_library','reggie','harbaugh','centraldiag','umma']

In [4]:
interact(display_style, x = style_suite)

interactive(children=(Dropdown(description='x', options=('bullfight', '2500_bullfight', '5000_bullfight', '100…

<function __main__.display_style(x)>

In [5]:
interact(display_content, x = content_suite)

interactive(children=(Dropdown(description='x', options=('law_library', 'reggie', 'harbaugh', 'centraldiag', '…

<function __main__.display_content(x)>

In [6]:
interact(stylize, style = style_suite, content = content_suite)

interactive(children=(Dropdown(description='style', options=('bullfight', '2500_bullfight', '5000_bullfight', …

<function __main__.stylize(style, content)>